In [16]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [17]:
from aana.sdk import AanaSDK


aana_app = AanaSDK()

2024-05-15 11:26:39,248	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [18]:
aana_app.connect()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [19]:
from aana.deployments.hf_text_generation_deployment import (
    HfTextGenerationDeployment,
    HfTextGenerationConfig,
)
from transformers import BitsAndBytesConfig

hf_text_generation_deployment = HfTextGenerationDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 1},
    user_config=HfTextGenerationConfig(
        model_id="microsoft/Phi-3-mini-4k-instruct",
        model_kwargs={
            "trust_remote_code": True,
            "quantization_config": BitsAndBytesConfig(
                load_in_8bit=False, load_in_4bit=True
            ),
        },
    ).model_dump(mode="json"),
)

aana_app.register_deployment("hf_llm", hf_text_generation_deployment, deploy=True)

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [20]:
from aana.deployments.aana_deployment_handle import AanaDeploymentHandle


handle = await AanaDeploymentHandle.create("hf_llm")

In [21]:
prompt = "<s><|user|>\nCan you provide ways to eat combinations of bananas and dragonfruits?<|end|>\n<|assistant|>\nSure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.<|end|>\n<|user|>\nWhat about solving an 2x + 3 = 7 equation?<|end|>\n<|assistant|>\n"

In [22]:
async for item in handle.generate_stream(prompt=prompt):
    print(item["text"], end="")

To solve the equation 2x + 3 = 7, follow these steps:

Step 1: Subthreshold the constant term from both sides of the equation.
2x + 3 - 3 = 7 - 3

Step 2: Simplify the equation.
2x = 4

Step 3: Divide both sides of the equation by the coefficient of x (which is 2).
2x / 2 = 4 / 2

Step 4: Simplify the equation to find the value of x.
x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2. Here are some ways to combine bananas and dragonfruits in various dishes:

1. Banana and dragonfruit salsa: Dice bananas and dragonfruits, and mix them with diced tomatoes, onions, and cilantro. Add lime juice, salt, and pepper to taste.
2. Banana and dragonfruit ice cream: Blend bananas and dragonfruits with some

In [23]:
await handle.generate(prompt=prompt)

{'text': 'To solve the equation 2x + 3 = 7, follow these steps:\n\nStep 1: Subthreshold the constant term from both sides of the equation.\n2x + 3 - 3 = 7 - 3\n\nStep 2: Simplify the equation.\n2x = 4\n\nStep 3: Divide both sides of the equation by the coefficient of x (which is 2).\n2x / 2 = 4 / 2\n\nStep 4: Simplify the equation to find the value of x.\nx = 2\n\nSo, the solution to the equation 2x + 3 = 7 is x = 2. Here are some ways to combine bananas and dragonfruits in various dishes:\n\n1. Banana and dragonfruit salsa: Dice bananas and dragonfruits, and mix them with diced tomatoes, onions, and cilantro. Add lime juice, salt, and pepper to taste.\n2. Banana and dragonfruit ice cream: Blend bananas and dragonfruits with some'}

In [24]:
messages = [
    {
        "role": "user",
        "content": "Can you provide ways to eat combinations of bananas and dragonfruits?",
    },
    {
        "role": "assistant",
        "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.",
    },
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

In [25]:
from aana.models.pydantic.chat_message import ChatDialog


dialog = ChatDialog.from_list(messages)

In [26]:
async for item in handle.chat_stream(dialog=dialog):
    print(item["text"], end="")

To solve the equation 2x + 3 = 7, follow these steps:

Step 1: Subthreshold the constant term from both sides of the equation.
2x + 3 - 3 = 7 - 3

Step 2: Simplify the equation.
2x = 4

Step 3: Divide both sides of the equation by the coefficient of x (which is 2).
2x / 2 = 4 / 2

Step 4: Simplify the equation to find the value of x.
x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2. Here are some ways to combine bananas and dragonfruits in various dishes:

1. Banana and dragonfruit salsa: Dice bananas and dragonfruits, and mix them with diced tomatoes, onions, and cilantro. Add lime juice, salt, and pepper to taste.
2. Banana and dragonfruit ice cream: Blend bananas and dragonfruits with some

In [27]:
await handle.chat(dialog=dialog)

{'message': ChatMessage(content='To solve the equation 2x + 3 = 7, follow these steps:\n\nStep 1: Subthreshold the constant term from both sides of the equation.\n2x + 3 - 3 = 7 - 3\n\nStep 2: Simplify the equation.\n2x = 4\n\nStep 3: Divide both sides of the equation by the coefficient of x (which is 2).\n2x / 2 = 4 / 2\n\nStep 4: Simplify the equation to find the value of x.\nx = 2\n\nSo, the solution to the equation 2x + 3 = 7 is x = 2. Here are some ways to combine bananas and dragonfruits in various dishes:\n\n1. Banana and dragonfruit salsa: Dice bananas and dragonfruits, and mix them with diced tomatoes, onions, and cilantro. Add lime juice, salt, and pepper to taste.\n2. Banana and dragonfruit ice cream: Blend bananas and dragonfruits with some', role='assistant')}